# FASTA File Analysis Script

This Python script processes a FASTA file to extract key biological information about DNA sequences. It performs the following tasks:

1. **FASTA File Parsing**: Reads sequences and their identifiers from a FASTA file.
2. **Sequence Length Analysis**: Determines the number of sequences in the file, identifies the longest and shortest sequences, and their corresponding identifiers.
3. **Open Reading Frame (ORF) Detection**: Identifies all ORFs in a specified forward reading frame (1, 2, or 3). It finds the longest ORF in the file and provides its sequence ID, length, and starting position.
4. **Repeat Sequence Analysis**: Finds substrings of a given length that appear multiple times in the sequences and determines the most frequent repeat.

## Usage
Modify the `file_path`, `reading_frame`, and `repeat_length` variables to specify:
- The FASTA file location.
- The reading frame (1, 2, or 3) for ORF analysis.
- The length of repeat sequences to be analyzed.

Run the script to generate insights about the DNA sequences, including ORF detection and repeat frequency analysis.


In [28]:
!pip install biopython

In [29]:
from Bio.Blast import NCBIWWW, NCBIXML  # Import necessary BLAST modules

In [30]:
from collections import defaultdict, Counter
import re

def read_fasta(file_path):
    """Reads a FASTA file and returns a dictionary with sequence IDs as keys and sequences as values."""
    sequences = {}
    with open(file_path, 'r') as f:
        seq_id = None
        seq_data = []
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                if seq_id:
                    sequences[seq_id] = ''.join(seq_data)
                seq_id = line[1:].split()[0]  # Extract ID
                seq_data = []
            else:
                seq_data.append(line)
        if seq_id:
            sequences[seq_id] = ''.join(seq_data)
    return sequences

def analyze_sequence_lengths(sequences):
    """Finds the shortest and longest sequences and their identifiers."""
    lengths = {seq_id: len(seq) for seq_id, seq in sequences.items()}
    min_length = min(lengths.values())
    max_length = max(lengths.values())
    shortest = [seq_id for seq_id, length in lengths.items() if length == min_length]
    longest = [seq_id for seq_id, length in lengths.items() if length == max_length]
    return lengths, shortest, longest, min_length, max_length

def find_orfs(sequence, reading_frame):
    """Finds all ORFs in a given sequence and reading frame."""
    start_codon = 'ATG'
    stop_codons = {'TAA', 'TAG', 'TGA'}
    orfs = []
    frame_offset = reading_frame - 1

    for i in range(frame_offset, len(sequence) - 2, 3):
        if sequence[i:i+3] == start_codon:
            for j in range(i, len(sequence) - 2, 3):
                codon = sequence[j:j+3]
                if codon in stop_codons:
                    orfs.append((i+1, j+3, j+3-i))  # Position in 1-based index
                    break
    return orfs

def analyze_orfs(sequences, reading_frame):
    """Finds the longest ORF in each sequence and the longest ORF in the file."""
    longest_orf = (None, 0, None)  # (seq_id, length, start_position)
    orf_data = {}

    for seq_id, seq in sequences.items():
        orfs = find_orfs(seq, reading_frame)
        if orfs:
            max_orf = max(orfs, key=lambda x: x[2])
            orf_data[seq_id] = max_orf
            if max_orf[2] > longest_orf[1]:
                longest_orf = (seq_id, max_orf[2], max_orf[0])

    return orf_data, longest_orf

def find_repeats(sequences, repeat_length):
    """Finds repeated substrings of a given length and their frequencies."""
    repeat_counts = Counter()

    for seq in sequences.values():
        for i in range(len(seq) - repeat_length + 1):
            repeat_counts[seq[i:i+repeat_length]] += 1

    max_repeat = max(repeat_counts.values(), default=0)
    most_frequent_repeats = [k for k, v in repeat_counts.items() if v == max_repeat]

    return repeat_counts, most_frequent_repeats, max_repeat

def main(file_path, reading_frame, repeat_length):
    sequences = read_fasta(file_path)
    num_records = len(sequences)

    lengths, shortest, longest, min_len, max_len = analyze_sequence_lengths(sequences)
    orf_data, longest_orf = analyze_orfs(sequences, reading_frame)
    repeat_counts, most_frequent_repeats, max_repeat = find_repeats(sequences, repeat_length)

    print(f"Number of records in file: {num_records}")
    print(f"Shortest sequence(s) ({min_len} bp): {shortest}")
    print(f"Longest sequence(s) ({max_len} bp): {longest}")

    print("\nLongest ORF in the file:")
    print(f"Sequence ID: {longest_orf[0]}, Length: {longest_orf[1]}, Start Position: {longest_orf[2]}")

    print("\nMost frequent repeats of length", repeat_length, ":")
    for repeat in most_frequent_repeats:
        print(f"{repeat}: {max_repeat} times")

In [31]:
if __name__ == "__main__":
    file_path = "dna2.fasta"  # Replace with actual file path
    reading_frame = 2  # Change to 2 or 3 as needed
    repeat_length = 7  # Adjust based on requirement
    main(file_path, reading_frame, repeat_length)

Number of records in file: 18
Shortest sequence(s) (115 bp): ['gi|142022655|gb|EQ086233.1|346']
Longest sequence(s) (4894 bp): ['gi|142022655|gb|EQ086233.1|255']

Longest ORF in the file:
Sequence ID: gi|142022655|gb|EQ086233.1|16, Length: 1458, Start Position: 3071

Most frequent repeats of length 7 :
CGCGCCG: 63 times


In [32]:
# Define the filename for the FASTA file
filename = "myseq.fa"

# Define the FASTA-formatted content with multiple DNA sequences
# Define the FASTA content that matches the sequences in the image
fasta_content = """>gi|733962926|gb|KP271020.1| Zaire ebolavirus isolate Ebola virus
H.sapiens-wt/COD/2014/Lomela-Lokolia9, complete genome
CATGCTACGGTGCTAAAAAGCTCCATAGTTGAGGACATCGTGTTTTAAATATAGTAGTTGCC
>gi|733962903|gb|KP271019.1| Zaire ebolavirus isolate Ebola virus
H.sapiens-wt/COD/2014/Lomela-Lokolia17, partial genome
CATGCTACGGTGCTAAAAAGCTCCATAGTTGAGGACATCGTGTTTTAAATATAGTAGTTGCC
>gi|733962878|gb|KP271018.1| Zaire ebolavirus isolate Ebola virus
H.sapiens-wt/COD/2014/Lomela-Lokolia6, complete genome
CATGCTACGGTGCTAAAAAGCTCCATAGTTGAGGACATCGTGTTTTAAATATAGTAGTTGCC
"""


# Open the file in write mode and save the sequences
with open(filename, "w") as fasta_file:
    fasta_file.write(fasta_content)

# Print success message
print(f"The FASTA file '{filename}' has been successfully created with sample DNA sequences.")


The FASTA file 'myseq.fa' has been successfully created with sample DNA sequences.


- Imports the NCBIWWW module from Bio.Blast, which provides functionality to perform BLAST queries using NCBI's online servers.
- "blastn" is used for nucleotide sequence comparisons.
Other options include "blastp" (for proteins), "blastx" (translated nucleotide vs protein), and "tblastn" (protein vs translated nucleotide).
- "nt" stands for Nucleotide collection database, which contains a comprehensive set of publicly available nucleotide sequences. Other databases include "nr" (non-redundant protein sequences), "swissprot" (curated protein sequences), and "refseq_rna" (reference RNA sequences).


In [33]:
fasta_string = open("myseq.fa").read()  # Read the sequence file content

result_handle = NCBIWWW.qblast("blastn", "nt", fasta_string)  # Perform a remote BLAST search using the specified program and database
